In [ ]:
### needs fixing when month changes; set last_complete_month, and make sure that xlsx's are same ###
### depends on iva.xlxs & iva_occupacy_*.xlsx ###

last_complete_month = '2020-10-31'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set()

In [ ]:
df = pd.read_excel('iva.xlsx',sheet_name='Utdataportalen',skiprows=1)

In [ ]:
def to_eng_month(m):
    class MonthError(Exception):
        pass
    
    if m == 'Januari' : return 'January'
    if m == 'Februari' : return 'February'
    if m == 'Mars' : return 'March'
    if m == 'April' : return 'April'
    if m == 'Maj' : return 'May'
    if m == 'Juni' : return 'June'
    if m == 'Juli' : return 'July'
    if m == 'Augusti' : return 'August'
    if m == 'September' : return 'September'
    if m == 'Oktober' : return 'October'
    if m == 'November' : return 'November'
    if m == 'December' : return 'December'
    
    raise MonthError('cant translate {}'.format(m))
    
df[['month','year']] = df['Månad'].str.split(' ',expand=True)

df['month'] = df['month'].apply(to_eng_month)
df

In [ ]:
latest_day = pd.to_datetime(df.iloc[-1,4])
latest_month = latest_day + pd.DateOffset(months=1)
latest_month

In [ ]:
idx = pd.date_range('2015-01-01',latest_month,freq='M')
idx

In [ ]:
df.index = idx
df

In [ ]:
#df.index = df.index.date


In [ ]:
df.plot(y='Antal vårdtillfällen',kind='bar',figsize=(18,12))

In [ ]:
df

In [ ]:
def read_beds(filename,sheetname):
    df = pd.read_excel(filename,sheet_name=sheetname,skiprows=1,
                        parse_dates=True,index_col=0)
    
    return df

In [ ]:
#### previous years ####
df_beds_15 = read_beds('iva_occupacy_15.xlsx','Utdataportalen')
df_beds_16 = read_beds('iva_occupacy_16.xlsx','Utdataportalen')
df_beds_17 = read_beds('iva_occupacy_17.xlsx','Utdataportalen')
df_beds_18 = read_beds('iva_occupacy_18.xlsx','Utdataportalen')
df_beds_19 = read_beds('iva_occupacy_19.xlsx','Utdataportalen')

#### update with 2020 fresh data

df_beds_YTD = pd.concat([df_beds_15,df_beds_16,df_beds_17,df_beds_18,df_beds_19])

# drop previous 2020 data
#df_beds_YTD = df_beds_YTD[~(df_beds_YTD.index.year==2020)]

df_beds_2020_YTD = read_beds('iva_occupacy_2020_YTD.xlsx','Utdataportalen')
df_beds_YTD = pd.concat([df_beds_YTD,df_beds_2020_YTD])

df_beds = df_beds_YTD
df_beds

In [ ]:
df_beds_daily_avg_per_month = df_beds.resample('M').mean()
df_beds_daily_avg_per_month

In [ ]:
#df = df.loc['2018-01-31' : last_complete_month]
df

In [ ]:
#df = (df.loc['2018-01-31' : last_complete_month]).copy()
df

In [ ]:
df['Platser'] = df_beds_daily_avg_per_month['Platser']
df

In [ ]:
last_date = df.index[-1]
last_date

In [ ]:
df

In [ ]:
print (df)

df['month'] = pd.Categorical(df['month'],['January','February','March','April','May','June',
                                         'July','August','September','October','November','December'])

pivot = df.pivot(index='year',columns='month',values='Antal vårdtillfällen')
pivot

In [ ]:
title = 'SWEDEN ICU transactions stacked by month,' \
' Jan 2018 - {} YTD\nDataSource : http://portal.icuregswe.org/utdata/sv/home'.format(last_date.date())

pivot.plot(kind='bar',stacked=True,figsize=(18,12),title=title)
plt.legend(loc='lower center')
plt.ylabel ('nr transactions')
plt.savefig('swe_iva_transactions_stacked.jpg',format='jpg')

In [ ]:
pivot2 = df.pivot(index='year',columns='month',values='Antal vårdtillfällen >48 tim')
pivot2

In [ ]:
title = 'SWEDEN ICU transactions > 48h, stacked by month,' \
' Jan 2015 - {} YTD\nDataSource : http://portal.icuregswe.org/utdata/sv/home'.format(last_date.date())

pivot2.plot(kind='bar',stacked=True,figsize=(18,12),title=title)
plt.ylabel('nr transactions')
plt.legend(loc='lower center')
plt.savefig('swe_iva_long_trans_stacked.jpg',format='jpg')

In [ ]:
title = 'SWEDEN number of ICU occupancies Jan 2018 -  {}'.format(last_date.date())
df.plot(y='Antal vårdtillfällen',figsize=(18,12),style='o--',title=title)

In [ ]:
df.plot(y='Antal personer',figsize=(18,12))

In [ ]:
df.head()

In [ ]:
df = df[['Antal vårdtillfällen','Antal vårdtillfällen >48 tim','Antal personer','Platser']]

In [ ]:
df.plot(y='Antal vårdtillfällen >48 tim',figsize=(18,12))

In [ ]:
title = 'SWEDEN ICU monthly statistics Jan 2018 - {}\nDataSource : http://portal.icuregswe.org/utdata/sv/home'.format(
    last_date.date())

axes = df.plot(y=['Antal vårdtillfällen','Antal vårdtillfällen >48 tim','Antal personer','Platser'],
        subplots=True,style=['ro--','go--','bo--','ko--'],title=title,
        figsize=(18,12))

axes[0].set_ylabel('Nr ICU transactions')
axes[1].set_ylabel('Nr ICU transactions')
axes[2].set_ylabel('Nr individuals')
axes[3].set_ylabel('Avg daily nr of beds')

legends = ['Nr ICU Transactions','Nr ICU Transactions, duration > 48h',
           'Nr individuals','Nr Beds occupied (daily avg)']

for n,ax in enumerate(axes):
    ax.legend([legends[n]])

plt.savefig('swe_iva_stats.jpg',format='jpg')

In [ ]:
df.describe()

In [ ]:
df_beds

In [ ]:
df_beds.describe()

In [ ]:
df_beds.loc['2020-03-31': '2020-05-31'].plot(figsize=(18,12))

In [ ]:
df

In [ ]:
df_sums = pd.DataFrame(df['Antal vårdtillfällen'] + df['Antal vårdtillfällen >48 tim'],
                       columns=['Total ICU Transactions'])
df_sums

In [ ]:
idx = df_sums.index.format(lambda x : x.strftime('%Y-%m-%d'))
idx = idx[1:]

df_sums['date'] = idx
title = 'SWE monthly all ICU transactions'
df_sums.set_index('date').plot(kind='bar',figsize=(18,12),title=title)
plt.savefig('swe_iva_combined_transactions.jpg',format='jpg')

In [ ]:
df_sums

In [ ]:
sums = df_sums.groupby([df_sums.index.year,df_sums.index.month]).sum()
sums

In [ ]:
mask = sums.index.get_level_values(1) < 11
sums[mask]

In [ ]:
sums_YTD = sums[mask].groupby(sums[mask].index.get_level_values(0)).sum()
sums_YTD

In [ ]:
title = 'SWEDEN ICU all transactions 2015 - 2020 for period January 1:st - {} {}:st'.format(
    pd.to_datetime(last_date).month_name(),last_date.date().day)

sums_YTD.plot(kind='bar',figsize=(18,12),title=title)
plt.savefig('swe_iva_all_trans_cum_ytd.jpg',format='jpg')

In [ ]:
monthly = sums.unstack()
monthly.columns = range(1,13)
monthly

In [ ]:
monthly.loc[:,:10].sum(axis=1)

In [ ]:
pd.to_datetime(last_date).month_name()

In [ ]:
df_beds

In [ ]:
df_beds_weekly_avg = df_beds.resample('w').mean()
df_beds_weekly_avg

In [ ]:
(df_beds_weekly_avg - df_beds_weekly_avg.shift())[:-1].plot(figsize=(18,12),style='o--')

In [ ]:
### yearly cumsum ...?
year = df_beds_weekly_avg.index.year
df_beds_weekly_avg.groupby(year).cumsum().plot()

In [ ]:
daily_beds = df_beds.resample('D').mean()
daily_beds

In [ ]:
(daily_beds - daily_beds.shift()).plot(figsize=(18,12),style='o--')

In [ ]:
(daily_beds - daily_beds.mean()).plot(figsize=(18,12),style='o--')